In [14]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import numpy as np
import pandas as pd
import torch
import torchvision
from torch import optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from sklearn.metrics import pairwise_distances
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm.notebook import tqdm
import os
import sys
import time
import torch.nn as nn
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from stream_generators.market_basket_dataset import MarketBasketDataset, BinaryEncodingTransform, RemoveItemsTransform
from utils.MatplotlibUtils import reduce_dims_and_plot
from network.auto_encoder import AutoEncoder
from losses.knn_loss import KNNLoss
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
from soft_decision_tree.sdt_model import SDT
from sklearn.metrics import davies_bouldin_score

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
k = 4
tree_depth = 6
device = 'cuda'
dataset_path = r"/mnt/qnap/ekosman/Groceries_dataset.csv"

## Create the market basket dataset and use one-hot encoding for items

In [21]:
dataset = MarketBasketDataset(dataset_path=dataset_path)

In [22]:
model = AutoEncoder(dataset.n_items, 50, 4).train().to(device)
epochs = 500
lr = 5e-3
batch_size = 512
log_every = 5

In [23]:
dataset.transform = torchvision.transforms.Compose([
    RemoveItemsTransform(p=0.5),
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)
dataset.target_transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)

In [24]:
model.train()
data_iter = torch.utils.data.DataLoader(dataset,
                                     batch_size=batch_size,
                                     shuffle=True,
                                     num_workers=1,
                                     pin_memory=True)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
scheduler = ReduceLROnPlateau(optimizer, 'min', verbose=True, factor=0.7, threshold=1e-4)
knn_crt = KNNLoss(k=k).to(device)
losses = []
alpha = 10/170
gamma = 2
for epoch in range(epochs):
    total_loss = 0
    for iteration, (batch, target) in enumerate(data_iter):
        batch = batch.to(device)
        target = target.to(device)
        outputs, iterm = model(batch, return_intermidiate=True)
        mse_loss = F.binary_cross_entropy_with_logits(outputs, target, reduction='none')
        mask = torch.ones_like(mse_loss)
        mask[target == 0] = alpha ** gamma
        mask[target == 1] = (1 - alpha) ** gamma
        mse_loss = (mse_loss * mask).sum(dim=-1).mean()
        try:
            knn_loss = knn_crt(iterm)
            if torch.isinf(knn_loss):
                knn_loss = 0
        except ValueError:
            knn_loss = torch.tensor(0)
        loss = mse_loss + knn_loss
        loss = mse_loss
        total_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if iteration % log_every == 0:
            print(f"Epoch {epoch} / {epochs} | iteration {iteration} / {len(data_iter)} | Total Loss: {loss.item()} | KNN Loss: {knn_loss.item()} | BCE Loss: {mse_loss.item()}")
    
    scheduler.step(total_loss / (iteration + 1))
    losses.append(total_loss / (iteration + 1))

Epoch 0 / 500 | iteration 0 / 30 | Total Loss: 1.9548475742340088 | KNN Loss: 6.224452495574951 | BCE Loss: 1.9548475742340088
Epoch 0 / 500 | iteration 5 / 30 | Total Loss: 1.9269031286239624 | KNN Loss: 6.223893642425537 | BCE Loss: 1.9269031286239624
Epoch 0 / 500 | iteration 10 / 30 | Total Loss: 1.930891513824463 | KNN Loss: 6.223770618438721 | BCE Loss: 1.930891513824463
Epoch 0 / 500 | iteration 15 / 30 | Total Loss: 1.908257246017456 | KNN Loss: 6.223718166351318 | BCE Loss: 1.908257246017456
Epoch 0 / 500 | iteration 20 / 30 | Total Loss: 1.9198890924453735 | KNN Loss: 6.224004745483398 | BCE Loss: 1.9198890924453735
Epoch 0 / 500 | iteration 25 / 30 | Total Loss: 1.941080093383789 | KNN Loss: 6.224161624908447 | BCE Loss: 1.941080093383789
Epoch 1 / 500 | iteration 0 / 30 | Total Loss: 1.9609971046447754 | KNN Loss: 6.224413871765137 | BCE Loss: 1.9609971046447754
Epoch 1 / 500 | iteration 5 / 30 | Total Loss: 1.9316774606704712 | KNN Loss: 6.2242889404296875 | BCE Loss: 1.93

Epoch 10 / 500 | iteration 25 / 30 | Total Loss: 1.1807327270507812 | KNN Loss: 6.2258710861206055 | BCE Loss: 1.1807327270507812
Epoch 11 / 500 | iteration 0 / 30 | Total Loss: 1.158316731452942 | KNN Loss: 6.226020336151123 | BCE Loss: 1.158316731452942
Epoch 11 / 500 | iteration 5 / 30 | Total Loss: 1.1760101318359375 | KNN Loss: 6.225729465484619 | BCE Loss: 1.1760101318359375
Epoch 11 / 500 | iteration 10 / 30 | Total Loss: 1.1324989795684814 | KNN Loss: 6.225797653198242 | BCE Loss: 1.1324989795684814
Epoch 11 / 500 | iteration 15 / 30 | Total Loss: 1.1305344104766846 | KNN Loss: 6.226112365722656 | BCE Loss: 1.1305344104766846
Epoch 11 / 500 | iteration 20 / 30 | Total Loss: 1.1277282238006592 | KNN Loss: 6.225693702697754 | BCE Loss: 1.1277282238006592
Epoch 11 / 500 | iteration 25 / 30 | Total Loss: 1.1142151355743408 | KNN Loss: 6.226020336151123 | BCE Loss: 1.1142151355743408
Epoch 12 / 500 | iteration 0 / 30 | Total Loss: 1.1002548933029175 | KNN Loss: 6.226152420043945 | B

Epoch 21 / 500 | iteration 15 / 30 | Total Loss: 1.0578217506408691 | KNN Loss: 6.225747108459473 | BCE Loss: 1.0578217506408691
Epoch 21 / 500 | iteration 20 / 30 | Total Loss: 1.0658769607543945 | KNN Loss: 6.225818634033203 | BCE Loss: 1.0658769607543945
Epoch 21 / 500 | iteration 25 / 30 | Total Loss: 1.0609794855117798 | KNN Loss: 6.225801467895508 | BCE Loss: 1.0609794855117798
Epoch 22 / 500 | iteration 0 / 30 | Total Loss: 1.0351464748382568 | KNN Loss: 6.22580099105835 | BCE Loss: 1.0351464748382568
Epoch 22 / 500 | iteration 5 / 30 | Total Loss: 1.0949946641921997 | KNN Loss: 6.2258172035217285 | BCE Loss: 1.0949946641921997
Epoch 22 / 500 | iteration 10 / 30 | Total Loss: 1.0600221157073975 | KNN Loss: 6.225919723510742 | BCE Loss: 1.0600221157073975
Epoch 22 / 500 | iteration 15 / 30 | Total Loss: 1.069531798362732 | KNN Loss: 6.225830078125 | BCE Loss: 1.069531798362732
Epoch 22 / 500 | iteration 20 / 30 | Total Loss: 1.0701208114624023 | KNN Loss: 6.225661277770996 | BCE 

Epoch 32 / 500 | iteration 5 / 30 | Total Loss: 1.0552157163619995 | KNN Loss: 6.225578308105469 | BCE Loss: 1.0552157163619995
Epoch 32 / 500 | iteration 10 / 30 | Total Loss: 1.0483157634735107 | KNN Loss: 6.225674629211426 | BCE Loss: 1.0483157634735107
Epoch 32 / 500 | iteration 15 / 30 | Total Loss: 1.036783218383789 | KNN Loss: 6.225553512573242 | BCE Loss: 1.036783218383789
Epoch 32 / 500 | iteration 20 / 30 | Total Loss: 1.0388524532318115 | KNN Loss: 6.225253105163574 | BCE Loss: 1.0388524532318115
Epoch 32 / 500 | iteration 25 / 30 | Total Loss: 1.0534021854400635 | KNN Loss: 6.225140571594238 | BCE Loss: 1.0534021854400635
Epoch 33 / 500 | iteration 0 / 30 | Total Loss: 1.070055603981018 | KNN Loss: 6.225834846496582 | BCE Loss: 1.070055603981018
Epoch 33 / 500 | iteration 5 / 30 | Total Loss: 1.0525712966918945 | KNN Loss: 6.225413799285889 | BCE Loss: 1.0525712966918945
Epoch 33 / 500 | iteration 10 / 30 | Total Loss: 1.0483182668685913 | KNN Loss: 6.225648880004883 | BCE 

Epoch 42 / 500 | iteration 25 / 30 | Total Loss: 1.0471668243408203 | KNN Loss: 6.225060939788818 | BCE Loss: 1.0471668243408203
Epoch 43 / 500 | iteration 0 / 30 | Total Loss: 1.0459469556808472 | KNN Loss: 6.225045204162598 | BCE Loss: 1.0459469556808472
Epoch 43 / 500 | iteration 5 / 30 | Total Loss: 1.0588915348052979 | KNN Loss: 6.225334644317627 | BCE Loss: 1.0588915348052979
Epoch 43 / 500 | iteration 10 / 30 | Total Loss: 1.03245210647583 | KNN Loss: 6.2253241539001465 | BCE Loss: 1.03245210647583
Epoch 43 / 500 | iteration 15 / 30 | Total Loss: 1.0404412746429443 | KNN Loss: 6.225056171417236 | BCE Loss: 1.0404412746429443
Epoch 43 / 500 | iteration 20 / 30 | Total Loss: 1.077479362487793 | KNN Loss: 6.225242614746094 | BCE Loss: 1.077479362487793
Epoch 43 / 500 | iteration 25 / 30 | Total Loss: 1.0450937747955322 | KNN Loss: 6.225175380706787 | BCE Loss: 1.0450937747955322
Epoch 44 / 500 | iteration 0 / 30 | Total Loss: 1.0388567447662354 | KNN Loss: 6.225160121917725 | BCE L

...

Epoch 454 / 500 | iteration 25 / 30 | Total Loss: 1.0386836528778076 | KNN Loss: 6.2243499755859375 | BCE Loss: 1.0386836528778076
Epoch 455 / 500 | iteration 0 / 30 | Total Loss: 1.0423836708068848 | KNN Loss: 6.224418640136719 | BCE Loss: 1.0423836708068848
Epoch 455 / 500 | iteration 5 / 30 | Total Loss: 1.0410542488098145 | KNN Loss: 6.2241291999816895 | BCE Loss: 1.0410542488098145
Epoch 455 / 500 | iteration 10 / 30 | Total Loss: 1.0504595041275024 | KNN Loss: 6.224740505218506 | BCE Loss: 1.0504595041275024
Epoch 455 / 500 | iteration 15 / 30 | Total Loss: 1.0450432300567627 | KNN Loss: 6.224768161773682 | BCE Loss: 1.0450432300567627
Epoch 455 / 500 | iteration 20 / 30 | Total Loss: 1.0475773811340332 | KNN Loss: 6.224482536315918 | BCE Loss: 1.0475773811340332
Epoch 455 / 500 | iteration 25 / 30 | Total Loss: 1.0611717700958252 | KNN Loss: 6.224587440490723 | BCE Loss: 1.0611717700958252
Epoch 456 / 500 | iteration 0 / 30 | Total Loss: 1.07692289352417 | KNN Loss: 6.2246189117

Epoch 465 / 500 | iteration 15 / 30 | Total Loss: 1.0223348140716553 | KNN Loss: 6.224613666534424 | BCE Loss: 1.0223348140716553
Epoch 465 / 500 | iteration 20 / 30 | Total Loss: 1.067919373512268 | KNN Loss: 6.224935531616211 | BCE Loss: 1.067919373512268
Epoch 465 / 500 | iteration 25 / 30 | Total Loss: 1.043061375617981 | KNN Loss: 6.224202632904053 | BCE Loss: 1.043061375617981
Epoch 466 / 500 | iteration 0 / 30 | Total Loss: 1.045883059501648 | KNN Loss: 6.2244791984558105 | BCE Loss: 1.045883059501648
Epoch 466 / 500 | iteration 5 / 30 | Total Loss: 1.0668728351593018 | KNN Loss: 6.224584102630615 | BCE Loss: 1.0668728351593018
Epoch 466 / 500 | iteration 10 / 30 | Total Loss: 1.0705169439315796 | KNN Loss: 6.224330425262451 | BCE Loss: 1.0705169439315796
Epoch 466 / 500 | iteration 15 / 30 | Total Loss: 1.0422401428222656 | KNN Loss: 6.224624156951904 | BCE Loss: 1.0422401428222656
Epoch 466 / 500 | iteration 20 / 30 | Total Loss: 1.044601559638977 | KNN Loss: 6.224548816680908

Epoch 476 / 500 | iteration 5 / 30 | Total Loss: 1.0375841856002808 | KNN Loss: 6.224062919616699 | BCE Loss: 1.0375841856002808
Epoch 476 / 500 | iteration 10 / 30 | Total Loss: 1.052534818649292 | KNN Loss: 6.224465847015381 | BCE Loss: 1.052534818649292
Epoch 476 / 500 | iteration 15 / 30 | Total Loss: 1.045093059539795 | KNN Loss: 6.224832057952881 | BCE Loss: 1.045093059539795
Epoch 476 / 500 | iteration 20 / 30 | Total Loss: 1.063495397567749 | KNN Loss: 6.224554538726807 | BCE Loss: 1.063495397567749
Epoch 476 / 500 | iteration 25 / 30 | Total Loss: 1.0082381963729858 | KNN Loss: 6.22435188293457 | BCE Loss: 1.0082381963729858
Epoch 477 / 500 | iteration 0 / 30 | Total Loss: 1.0363152027130127 | KNN Loss: 6.224386215209961 | BCE Loss: 1.0363152027130127
Epoch 477 / 500 | iteration 5 / 30 | Total Loss: 1.0466156005859375 | KNN Loss: 6.224661350250244 | BCE Loss: 1.0466156005859375
Epoch 477 / 500 | iteration 10 / 30 | Total Loss: 1.0606563091278076 | KNN Loss: 6.22452449798584 | 

Epoch 486 / 500 | iteration 25 / 30 | Total Loss: 1.055832862854004 | KNN Loss: 6.22460412979126 | BCE Loss: 1.055832862854004
Epoch 487 / 500 | iteration 0 / 30 | Total Loss: 1.053954839706421 | KNN Loss: 6.224515438079834 | BCE Loss: 1.053954839706421
Epoch 487 / 500 | iteration 5 / 30 | Total Loss: 1.0651004314422607 | KNN Loss: 6.224644660949707 | BCE Loss: 1.0651004314422607
Epoch 487 / 500 | iteration 10 / 30 | Total Loss: 1.057348370552063 | KNN Loss: 6.224420547485352 | BCE Loss: 1.057348370552063
Epoch 487 / 500 | iteration 15 / 30 | Total Loss: 1.0303072929382324 | KNN Loss: 6.224255084991455 | BCE Loss: 1.0303072929382324
Epoch 487 / 500 | iteration 20 / 30 | Total Loss: 1.054123878479004 | KNN Loss: 6.224785327911377 | BCE Loss: 1.054123878479004
Epoch 487 / 500 | iteration 25 / 30 | Total Loss: 1.0397799015045166 | KNN Loss: 6.224265098571777 | BCE Loss: 1.0397799015045166
Epoch 488 / 500 | iteration 0 / 30 | Total Loss: 1.0715899467468262 | KNN Loss: 6.2243733406066895 | 

Epoch 497 / 500 | iteration 15 / 30 | Total Loss: 1.0636194944381714 | KNN Loss: 6.224575996398926 | BCE Loss: 1.0636194944381714
Epoch 497 / 500 | iteration 20 / 30 | Total Loss: 1.0476899147033691 | KNN Loss: 6.2240729331970215 | BCE Loss: 1.0476899147033691
Epoch 497 / 500 | iteration 25 / 30 | Total Loss: 1.036081314086914 | KNN Loss: 6.224581241607666 | BCE Loss: 1.036081314086914
Epoch 498 / 500 | iteration 0 / 30 | Total Loss: 1.066345453262329 | KNN Loss: 6.224164009094238 | BCE Loss: 1.066345453262329
Epoch 498 / 500 | iteration 5 / 30 | Total Loss: 1.0668036937713623 | KNN Loss: 6.224806785583496 | BCE Loss: 1.0668036937713623
Epoch 498 / 500 | iteration 10 / 30 | Total Loss: 1.0526536703109741 | KNN Loss: 6.224017143249512 | BCE Loss: 1.0526536703109741
Epoch 498 / 500 | iteration 15 / 30 | Total Loss: 1.026498794555664 | KNN Loss: 6.224737167358398 | BCE Loss: 1.026498794555664
Epoch 498 / 500 | iteration 20 / 30 | Total Loss: 1.048725962638855 | KNN Loss: 6.22463321685791 

In [25]:
outputs, iterm = model(dataset[67][0].unsqueeze(0).to(device), return_intermidiate=True)
print(outputs)
print(outputs.min())
print(outputs.max())

tensor([[ 2.9231,  3.8722,  2.5812,  3.5745,  3.4543,  0.7017,  2.6651,  2.1970,
          2.3083,  1.9925,  2.2344,  2.1979,  0.7876,  1.8203,  1.2891,  1.5216,
          2.8064,  3.1788,  2.7991,  2.3030,  1.7450,  2.9516,  2.2904,  2.6379,
          2.5332,  1.7390,  2.1229,  1.4155,  1.4934,  0.3250, -0.2396,  0.9983,
          0.2212,  0.9240,  1.5308,  1.4778,  1.0049,  3.3144,  0.8001,  1.3198,
          0.9675, -0.7032, -0.2357,  2.3364,  2.1887,  0.7335, -0.2012,  0.0970,
          1.4607,  2.4952,  1.8230,  0.1381,  1.4258,  0.5204, -0.6363,  1.1095,
          1.4810,  1.3720,  1.3400,  1.8270,  0.5738,  0.8433,  0.1389,  1.7235,
          1.3158,  1.6659, -1.8257,  0.3069,  2.2878,  2.1419,  2.5500,  0.4267,
          1.3524,  2.4587,  1.9961,  1.2923,  0.2264,  0.7375,  0.2173,  1.5898,
          0.0257,  0.3779,  1.8381, -0.3734,  0.2388, -1.0732, -2.2649, -0.2561,
          0.5456, -1.8580,  0.4615, -0.1291, -0.5688, -0.9364,  0.5635,  1.2683,
         -0.6937, -0.7033,  

In [26]:
plt.figure()
plt.plot(losses)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [27]:
dataset.transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)
dataset.target_transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)

In [28]:
dataset_ = [d[0].cpu() for d in dataset]

In [29]:
model = model.eval().cpu()
projections = model.calculate_intermidiate(dataset_)

100%|██████████| 15/15 [00:00<00:00, 72.73it/s]


In [30]:
distances = pairwise_distances(projections)
# distances = np.triu(distances)
distances_f = distances.flatten()

plt.matshow(distances)
plt.colorbar()
plt.figure()
plt.hist(distances_f[distances_f > 0], bins=1000)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Fit DBSCAN and calculate indices


In [107]:
clusters = DBSCAN(eps=0.01, min_samples=80).fit_predict(projections)
# scores = []
# best_score = float('inf')
# clusters = None
# range_ = list(range(5, 20))
# for k in tqdm(range_):
#     y = GaussianMixture(n_components=k).fit_predict(projections)
#     cur_score = davies_bouldin_score(projections, y)
#     scores.append(cur_score)
    
#     if cur_score < best_score:
#         best_score = cur_score
#         clusters = y

In [108]:
perplexity = 100

p = reduce_dims_and_plot(projections[clusters != -1],
                         y=clusters[clusters != -1],
                         title=f'perplexity: {perplexity}',
                         file_name=None,
                         perplexity=perplexity,
                         library='Multicore-TSNE',
                         perform_PCA=False,
                         projected=None,
                         figure_type='2d',
                         show_figure=True,
                         close_figure=False,
                         text=None)    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [109]:
# from sklearn.tree import DecisionTreeClassifier
# from sklearn import tree
# from sklearn.tree import _tree

In [110]:
tensor_dataset = torch.stack(dataset_)

In [111]:
# clf = DecisionTreeClassifier(max_depth=200, min_samples_leaf=5)
# clf = clf.fit(tensor_dataset[clusters!=-1], clusters[clusters != -1])
# print(clf.score(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
# print(clf.get_depth())

In [112]:
# scores = []
# for min_samples in range(1,50, 1):
#     clf = DecisionTreeClassifier(max_depth=200, min_samples_leaf=min_samples)
#     clf = clf.fit(tensor_dataset[clusters!=-1], clusters[clusters != -1])
#     scores.append(clf.score(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
    
# plt.figure()
# plt.plot(list(range(1,50, 1)), scores)
# plt.show()

In [113]:
def get_rules(tree, feature_names, class_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]

    paths = []
    path = []
    
    def recurse(node, path, paths):
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            p1, p2 = list(path), list(path)
#             p1 += [f"({name} <= {np.round(threshold, 3)})"]
            p1 += [(name, '<=', np.round(threshold, 3))]
            recurse(tree_.children_left[node], p1, paths)
            p2 += [(name, '>', np.round(threshold, 3))]
            recurse(tree_.children_right[node], p2, paths)
        else:
            path += [(tree_.value[node], tree_.n_node_samples[node])]
            paths += [path]
            
    recurse(0, path, paths)

    # sort by samples count
    samples_count = [p[-1][1] for p in paths]
    ii = list(np.argsort(samples_count))
    paths = [paths[i] for i in reversed(ii)]
    
    rules = []
    for path in paths:
        rule = []
        
        for p in path[:-1]:
            rule += [p]
        target = " then "
        if class_names is None:
            target += "response: "+str(np.round(path[-1][0][0][0],3))
        else:
            classes = path[-1][0][0]
            l = np.argmax(classes)
            target += f"class: {class_names[l]} (proba: {np.round(100.0*classes[l]/np.sum(classes),2)}%)"
           
        proba = np.round(100.0*classes[l]/np.sum(classes),2)
        target += f" | based on {path[-1][1]:,} samples"
        rule_wrapper = {'target': target, 'rule': rule, 'proba': proba}
        rules += [rule_wrapper]
        
    return rules

In [114]:
# rules = get_rules(clf, dataset.items, clusters[clusters != -1])

# for rule in rules:
#     n_pos = 0
#     for c,p,v in rule['rule']:
#         if p == '>':
#             n_pos += 1
#     rule['pos'] = n_pos

In [115]:
# plt.figure()
# probs = [r['proba'] for r in rules]
# plt.hist(probs, bins = 100)
# plt.show()

In [116]:
# rules = sorted(rules, key=lambda x:x['pos'])
# rules = [r for r in rules if r['proba'] > 50]
# print(len(rules))

In [117]:
# for i in range(17):
#     r_i = rules[i]
#     print(f"------------- rule {i} length {len(r_i)} -------------")
#     print(r_i)

# Train a Soft-Decision-Tree given the self-labels

## Prepare the dataset

In [118]:
tree_dataset = list(zip(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
batch_size = 512
tree_loader = torch.utils.data.DataLoader(tree_dataset, batch_size=batch_size, shuffle=True)

# Define how we prune the weights of a node

In [119]:
def prune_node(node_weights, factor=1):
    w = node_weights.cpu().detach().numpy()
    mean_ = np.mean(w)
    std_ = np.std(w)
    node_weights[((mean_ - std_ * factor) < node_weights) & (node_weights < (mean_ + std_ * factor))] = 0
    return node_weights

def prune_node_keep(node_weights, keep=4):
    w = node_weights.cpu().detach().numpy()
    throw_idx = np.argsort(abs(w))[:-keep]
    node_weights[throw_idx] = 0
    return node_weights

def prune_tree(tree_, factor):
    new_weights = tree_.inner_nodes.weight.clone()
    for i in range(new_weights.shape[0]):
        res = prune_node_keep(new_weights[i, :], factor)
        new_weights[i, :] = res

    with torch.no_grad():
        tree_.inner_nodes.weight.copy_(new_weights)
        
def sparseness(x):
    s = []
    for i in range(x.shape[0]):
        x_ = x[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        s.append(sp)
    return np.mean(s)

def compute_regularization_by_level(tree):
    total_reg = 0
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_level = np.floor(np.log2(i+1))
        node_reg = torch.norm(tree.inner_nodes.weight[i].view(-1), 2)
        total_reg += 2**(-cur_level) * node_reg
    return total_reg

def show_sparseness(tree):
    avg_sp = sparseness(tree.inner_nodes.weight)
    print(f"Average sparseness: {avg_sp}")
    layer = 0
    sps = []
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_layer = int(np.floor(np.log2(i+1)))
        if cur_layer != layer:
            print(f"layer {layer}: {np.mean(sps)}")
            sps = []
            layer = cur_layer

        x_ = tree.inner_nodes.weight[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        sps.append(sp)
        
    return avg_sp

## Training configurations

In [120]:
def do_epoch(model, loader, device, log_interval, losses, accs, epoch, iteration):
    model = model.train()
    for batch_idx, (data, target) in enumerate(loader):
        iteration += 1
        data, target = data.to(device), target.to(device)

        output, penalty = tree.forward(data)

        # Loss
        loss_tree = criterion(output, target.view(-1))

        # Penalty
        loss_tree += penalty

        # Sparse regularization
#         fc_params = torch.cat([x.view(-1) for x in tree.inner_nodes.parameters()])
#         regularization = sparsity_lamda * torch.norm(fc_params, 2)
        regularization = sparsity_lamda * compute_regularization_by_level(tree)
        loss = loss_tree

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(loss.item())

        pred = output.data.max(1)[1]
        correct = pred.eq(target.view(-1).data).sum()
        accs.append(correct.item() / data.size()[0])

        # Print training status
        if batch_idx % log_interval == 0:
            print(f"Epoch: {epoch:02d} | Batch: {batch_idx:03d} / {len(loader):03d} | Total loss: {loss.item():.3f} | Reg loss: {regularization.item():.3f} | Tree loss: {loss_tree.item():.3f} | Accuracy: {correct.item() / data.size()[0]:03f} | {round((time.time() - start_time) / iteration, 3)} sec/iter")
            
    return iteration


In [121]:
lr = 5e-3
weight_decay = 5e-4
sparsity_lamda = 2e-3
epochs = 100
output_dim = len(set(clusters))
log_interval = 1
use_cuda = device != 'cpu'

In [122]:
tree = SDT(input_dim=tensor_dataset.shape[1], output_dim=len(clusters - 1), depth=tree_depth, lamda=1e-3, use_cuda=use_cuda)
optimizer = torch.optim.Adam(tree.parameters(),
                                 lr=lr,
                                 weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()
tree = tree.to(device)

In [123]:
losses = []
accs = []
sparsity = []

In [124]:
start_time = time.time()
iteration = 0
for epoch in range(epochs):
    # Training
    avg_sp = show_sparseness(tree)
    sparsity.append(avg_sp)
    iteration = do_epoch(tree, tree_loader, device, log_interval, losses, accs, epoch, iteration)
    
    if epoch % 1 == 0:
        prune_tree(tree, factor=3)
        

Average sparseness: 0.0
layer 0: 0.0
layer 1: 0.0
layer 2: 0.0
layer 3: 0.0
layer 4: 0.0
Epoch: 00 | Batch: 000 / 025 | Total loss: 9.612 | Reg loss: 0.007 | Tree loss: 9.612 | Accuracy: 0.000000 | 0.095 sec/iter
Epoch: 00 | Batch: 001 / 025 | Total loss: 9.592 | Reg loss: 0.007 | Tree loss: 9.592 | Accuracy: 0.000000 | 0.082 sec/iter
Epoch: 00 | Batch: 002 / 025 | Total loss: 9.572 | Reg loss: 0.007 | Tree loss: 9.572 | Accuracy: 0.000000 | 0.08 sec/iter
Epoch: 00 | Batch: 003 / 025 | Total loss: 9.553 | Reg loss: 0.007 | Tree loss: 9.553 | Accuracy: 0.000000 | 0.076 sec/iter
Epoch: 00 | Batch: 004 / 025 | Total loss: 9.531 | Reg loss: 0.007 | Tree loss: 9.531 | Accuracy: 0.000000 | 0.073 sec/iter
Epoch: 00 | Batch: 005 / 025 | Total loss: 9.512 | Reg loss: 0.007 | Tree loss: 9.512 | Accuracy: 0.000000 | 0.071 sec/iter
Epoch: 00 | Batch: 006 / 025 | Total loss: 9.492 | Reg loss: 0.007 | Tree loss: 9.492 | Accuracy: 0.000000 | 0.069 sec/iter
Epoch: 00 | Batch: 007 / 025 | Total loss: 9

Epoch: 02 | Batch: 015 / 025 | Total loss: 9.022 | Reg loss: 0.010 | Tree loss: 9.022 | Accuracy: 1.000000 | 0.066 sec/iter
Epoch: 02 | Batch: 016 / 025 | Total loss: 9.010 | Reg loss: 0.011 | Tree loss: 9.010 | Accuracy: 1.000000 | 0.066 sec/iter
Epoch: 02 | Batch: 017 / 025 | Total loss: 8.991 | Reg loss: 0.011 | Tree loss: 8.991 | Accuracy: 1.000000 | 0.066 sec/iter
Epoch: 02 | Batch: 018 / 025 | Total loss: 8.972 | Reg loss: 0.011 | Tree loss: 8.972 | Accuracy: 1.000000 | 0.066 sec/iter
Epoch: 02 | Batch: 019 / 025 | Total loss: 8.947 | Reg loss: 0.012 | Tree loss: 8.947 | Accuracy: 1.000000 | 0.066 sec/iter
Epoch: 02 | Batch: 020 / 025 | Total loss: 8.933 | Reg loss: 0.012 | Tree loss: 8.933 | Accuracy: 1.000000 | 0.066 sec/iter
Epoch: 02 | Batch: 021 / 025 | Total loss: 8.911 | Reg loss: 0.013 | Tree loss: 8.911 | Accuracy: 1.000000 | 0.067 sec/iter
Epoch: 02 | Batch: 022 / 025 | Total loss: 8.882 | Reg loss: 0.013 | Tree loss: 8.882 | Accuracy: 1.000000 | 0.066 sec/iter
Epoch: 0

Epoch: 05 | Batch: 004 / 025 | Total loss: 8.814 | Reg loss: 0.012 | Tree loss: 8.814 | Accuracy: 1.000000 | 0.067 sec/iter
Epoch: 05 | Batch: 005 / 025 | Total loss: 8.799 | Reg loss: 0.012 | Tree loss: 8.799 | Accuracy: 1.000000 | 0.067 sec/iter
Epoch: 05 | Batch: 006 / 025 | Total loss: 8.774 | Reg loss: 0.013 | Tree loss: 8.774 | Accuracy: 1.000000 | 0.067 sec/iter
Epoch: 05 | Batch: 007 / 025 | Total loss: 8.757 | Reg loss: 0.013 | Tree loss: 8.757 | Accuracy: 1.000000 | 0.067 sec/iter
Epoch: 05 | Batch: 008 / 025 | Total loss: 8.737 | Reg loss: 0.013 | Tree loss: 8.737 | Accuracy: 1.000000 | 0.067 sec/iter
Epoch: 05 | Batch: 009 / 025 | Total loss: 8.723 | Reg loss: 0.013 | Tree loss: 8.723 | Accuracy: 1.000000 | 0.067 sec/iter
Epoch: 05 | Batch: 010 / 025 | Total loss: 8.691 | Reg loss: 0.014 | Tree loss: 8.691 | Accuracy: 1.000000 | 0.067 sec/iter
Epoch: 05 | Batch: 011 / 025 | Total loss: 8.679 | Reg loss: 0.014 | Tree loss: 8.679 | Accuracy: 1.000000 | 0.067 sec/iter
Epoch: 0

Epoch: 07 | Batch: 019 / 025 | Total loss: 8.229 | Reg loss: 0.020 | Tree loss: 8.229 | Accuracy: 1.000000 | 0.067 sec/iter
Epoch: 07 | Batch: 020 / 025 | Total loss: 8.218 | Reg loss: 0.021 | Tree loss: 8.218 | Accuracy: 1.000000 | 0.067 sec/iter
Epoch: 07 | Batch: 021 / 025 | Total loss: 8.197 | Reg loss: 0.021 | Tree loss: 8.197 | Accuracy: 1.000000 | 0.067 sec/iter
Epoch: 07 | Batch: 022 / 025 | Total loss: 8.172 | Reg loss: 0.021 | Tree loss: 8.172 | Accuracy: 1.000000 | 0.067 sec/iter
Epoch: 07 | Batch: 023 / 025 | Total loss: 8.163 | Reg loss: 0.022 | Tree loss: 8.163 | Accuracy: 1.000000 | 0.067 sec/iter
Epoch: 07 | Batch: 024 / 025 | Total loss: 8.136 | Reg loss: 0.022 | Tree loss: 8.136 | Accuracy: 1.000000 | 0.067 sec/iter
Average sparseness: 0.9821428571428567
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
Epoch: 08 | Batch: 000 / 025 | Total loss: 8.483 | Reg loss: 0.017 | Tree los

Epoch: 10 | Batch: 006 / 025 | Total loss: 8.088 | Reg loss: 0.020 | Tree loss: 8.088 | Accuracy: 1.000000 | 0.067 sec/iter
Epoch: 10 | Batch: 007 / 025 | Total loss: 8.066 | Reg loss: 0.020 | Tree loss: 8.066 | Accuracy: 1.000000 | 0.067 sec/iter
Epoch: 10 | Batch: 008 / 025 | Total loss: 8.040 | Reg loss: 0.020 | Tree loss: 8.040 | Accuracy: 1.000000 | 0.067 sec/iter
Epoch: 10 | Batch: 009 / 025 | Total loss: 8.032 | Reg loss: 0.021 | Tree loss: 8.032 | Accuracy: 1.000000 | 0.067 sec/iter
Epoch: 10 | Batch: 010 / 025 | Total loss: 7.996 | Reg loss: 0.021 | Tree loss: 7.996 | Accuracy: 1.000000 | 0.067 sec/iter
Epoch: 10 | Batch: 011 / 025 | Total loss: 7.989 | Reg loss: 0.021 | Tree loss: 7.989 | Accuracy: 1.000000 | 0.067 sec/iter
Epoch: 10 | Batch: 012 / 025 | Total loss: 7.953 | Reg loss: 0.021 | Tree loss: 7.953 | Accuracy: 1.000000 | 0.067 sec/iter
Epoch: 10 | Batch: 013 / 025 | Total loss: 7.943 | Reg loss: 0.022 | Tree loss: 7.943 | Accuracy: 1.000000 | 0.067 sec/iter
Epoch: 1

...

Epoch: 89 | Batch: 008 / 025 | Total loss: 0.389 | Reg loss: 0.024 | Tree loss: 0.389 | Accuracy: 1.000000 | 0.068 sec/iter
Epoch: 89 | Batch: 009 / 025 | Total loss: 0.384 | Reg loss: 0.024 | Tree loss: 0.384 | Accuracy: 1.000000 | 0.068 sec/iter
Epoch: 89 | Batch: 010 / 025 | Total loss: 0.372 | Reg loss: 0.024 | Tree loss: 0.372 | Accuracy: 1.000000 | 0.068 sec/iter
Epoch: 89 | Batch: 011 / 025 | Total loss: 0.371 | Reg loss: 0.024 | Tree loss: 0.371 | Accuracy: 1.000000 | 0.068 sec/iter
Epoch: 89 | Batch: 012 / 025 | Total loss: 0.368 | Reg loss: 0.024 | Tree loss: 0.368 | Accuracy: 1.000000 | 0.068 sec/iter
Epoch: 89 | Batch: 013 / 025 | Total loss: 0.363 | Reg loss: 0.024 | Tree loss: 0.363 | Accuracy: 1.000000 | 0.068 sec/iter
Epoch: 89 | Batch: 014 / 025 | Total loss: 0.369 | Reg loss: 0.024 | Tree loss: 0.369 | Accuracy: 1.000000 | 0.068 sec/iter
Epoch: 89 | Batch: 015 / 025 | Total loss: 0.360 | Reg loss: 0.024 | Tree loss: 0.360 | Accuracy: 1.000000 | 0.068 sec/iter
Epoch: 8

Epoch: 91 | Batch: 023 / 025 | Total loss: 0.317 | Reg loss: 0.024 | Tree loss: 0.317 | Accuracy: 1.000000 | 0.068 sec/iter
Epoch: 91 | Batch: 024 / 025 | Total loss: 0.310 | Reg loss: 0.024 | Tree loss: 0.310 | Accuracy: 1.000000 | 0.068 sec/iter
Average sparseness: 0.9821428571428567
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
Epoch: 92 | Batch: 000 / 025 | Total loss: 0.377 | Reg loss: 0.023 | Tree loss: 0.377 | Accuracy: 1.000000 | 0.068 sec/iter
Epoch: 92 | Batch: 001 / 025 | Total loss: 0.373 | Reg loss: 0.023 | Tree loss: 0.373 | Accuracy: 1.000000 | 0.068 sec/iter
Epoch: 92 | Batch: 002 / 025 | Total loss: 0.370 | Reg loss: 0.023 | Tree loss: 0.370 | Accuracy: 1.000000 | 0.068 sec/iter
Epoch: 92 | Batch: 003 / 025 | Total loss: 0.361 | Reg loss: 0.023 | Tree loss: 0.361 | Accuracy: 1.000000 | 0.068 sec/iter
Epoch: 92 | Batch: 004 / 025 | Total loss: 0.361 | Reg loss: 0.023 | Tree los

Epoch: 94 | Batch: 012 / 025 | Total loss: 0.319 | Reg loss: 0.023 | Tree loss: 0.319 | Accuracy: 1.000000 | 0.068 sec/iter
Epoch: 94 | Batch: 013 / 025 | Total loss: 0.320 | Reg loss: 0.023 | Tree loss: 0.320 | Accuracy: 1.000000 | 0.068 sec/iter
Epoch: 94 | Batch: 014 / 025 | Total loss: 0.318 | Reg loss: 0.023 | Tree loss: 0.318 | Accuracy: 1.000000 | 0.068 sec/iter
Epoch: 94 | Batch: 015 / 025 | Total loss: 0.310 | Reg loss: 0.024 | Tree loss: 0.310 | Accuracy: 1.000000 | 0.068 sec/iter
Epoch: 94 | Batch: 016 / 025 | Total loss: 0.313 | Reg loss: 0.024 | Tree loss: 0.313 | Accuracy: 1.000000 | 0.068 sec/iter
Epoch: 94 | Batch: 017 / 025 | Total loss: 0.306 | Reg loss: 0.024 | Tree loss: 0.306 | Accuracy: 1.000000 | 0.068 sec/iter
Epoch: 94 | Batch: 018 / 025 | Total loss: 0.305 | Reg loss: 0.024 | Tree loss: 0.305 | Accuracy: 1.000000 | 0.068 sec/iter
Epoch: 94 | Batch: 019 / 025 | Total loss: 0.303 | Reg loss: 0.024 | Tree loss: 0.303 | Accuracy: 1.000000 | 0.068 sec/iter
Epoch: 9

Epoch: 97 | Batch: 001 / 025 | Total loss: 0.325 | Reg loss: 0.023 | Tree loss: 0.325 | Accuracy: 1.000000 | 0.068 sec/iter
Epoch: 97 | Batch: 002 / 025 | Total loss: 0.322 | Reg loss: 0.023 | Tree loss: 0.322 | Accuracy: 1.000000 | 0.068 sec/iter
Epoch: 97 | Batch: 003 / 025 | Total loss: 0.321 | Reg loss: 0.023 | Tree loss: 0.321 | Accuracy: 1.000000 | 0.068 sec/iter
Epoch: 97 | Batch: 004 / 025 | Total loss: 0.316 | Reg loss: 0.023 | Tree loss: 0.316 | Accuracy: 1.000000 | 0.068 sec/iter
Epoch: 97 | Batch: 005 / 025 | Total loss: 0.315 | Reg loss: 0.023 | Tree loss: 0.315 | Accuracy: 1.000000 | 0.068 sec/iter
Epoch: 97 | Batch: 006 / 025 | Total loss: 0.313 | Reg loss: 0.023 | Tree loss: 0.313 | Accuracy: 1.000000 | 0.068 sec/iter
Epoch: 97 | Batch: 007 / 025 | Total loss: 0.313 | Reg loss: 0.023 | Tree loss: 0.313 | Accuracy: 1.000000 | 0.068 sec/iter
Epoch: 97 | Batch: 008 / 025 | Total loss: 0.306 | Reg loss: 0.023 | Tree loss: 0.306 | Accuracy: 1.000000 | 0.068 sec/iter
Epoch: 9

Epoch: 99 | Batch: 016 / 025 | Total loss: 0.276 | Reg loss: 0.023 | Tree loss: 0.276 | Accuracy: 1.000000 | 0.068 sec/iter
Epoch: 99 | Batch: 017 / 025 | Total loss: 0.272 | Reg loss: 0.023 | Tree loss: 0.272 | Accuracy: 1.000000 | 0.068 sec/iter
Epoch: 99 | Batch: 018 / 025 | Total loss: 0.271 | Reg loss: 0.023 | Tree loss: 0.271 | Accuracy: 1.000000 | 0.068 sec/iter
Epoch: 99 | Batch: 019 / 025 | Total loss: 0.267 | Reg loss: 0.023 | Tree loss: 0.267 | Accuracy: 1.000000 | 0.068 sec/iter
Epoch: 99 | Batch: 020 / 025 | Total loss: 0.264 | Reg loss: 0.023 | Tree loss: 0.264 | Accuracy: 1.000000 | 0.068 sec/iter
Epoch: 99 | Batch: 021 / 025 | Total loss: 0.261 | Reg loss: 0.023 | Tree loss: 0.261 | Accuracy: 1.000000 | 0.068 sec/iter
Epoch: 99 | Batch: 022 / 025 | Total loss: 0.263 | Reg loss: 0.023 | Tree loss: 0.263 | Accuracy: 1.000000 | 0.068 sec/iter
Epoch: 99 | Batch: 023 / 025 | Total loss: 0.262 | Reg loss: 0.023 | Tree loss: 0.262 | Accuracy: 1.000000 | 0.068 sec/iter
Epoch: 9

In [125]:
plt.figure(figsize=(10, 5))
plt.ylabel("Accuracy")
plt.xlabel('Iteration')
plt.plot(accs, label='Accuracy vs iteration')
plt.show()

/home/eitan.k/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [126]:
plt.figure()
plt.ylabel("Loss")
plt.xlabel('Iteration')
plt.plot(losses, label='Loss vs iteration')
plt.yscale("log")
plt.show()

plt.figure()
weights = tree.inner_nodes.weight.cpu().detach().numpy().flatten()
plt.hist(weights, bins=500)
weights_std = np.std(weights)
weights_mean = np.mean(weights)
plt.axvline(weights_mean + weights_std, color='r')
plt.axvline(weights_mean - weights_std, color='r')
plt.title(f"Mean: {weights_mean}   |   STD: {weights_std}")
plt.yscale("log")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Tree Visualization

In [127]:
plt.figure(figsize=(15, 10), dpi=80)
avg_height, root = tree.visualize()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Average height: 0.0


# Extract Rules

# Accumulate samples in the leaves

In [128]:
print(f"Number of patterns: {len(root.get_leaves())}")

Number of patterns: 1


In [129]:
method = 'greedy'

In [130]:
root.clear_leaves_samples()

with torch.no_grad():
    for batch_idx, (data, target) in enumerate(tree_loader):
        root.accumulate_samples(data, method)

# Tighten boundaries

In [131]:
attr_names = dataset.items

# print(attr_names)
leaves = root.get_leaves()
sum_comprehensibility = 0
comprehensibilities = []
for pattern_counter, leaf in enumerate(leaves):
    leaf.reset_path()
    leaf.tighten_with_accumulated_samples()
    conds = leaf.get_path_conditions(attr_names)
    for cond in conds:
        cond.weights = cond.weights / normalizers
    print(f"============== Pattern {pattern_counter + 1} ==============")
    comprehensibilities.append(sum([cond.comprehensibility for cond in conds]))
    
print(f"Average comprehensibility: {np.mean(comprehensibilities)}")
print(f"std comprehensibility: {np.std(comprehensibilities)}")
print(f"var comprehensibility: {np.var(comprehensibilities)}")
print(f"minimum comprehensibility: {np.min(comprehensibilities)}")
print(f"maximum comprehensibility: {np.max(comprehensibilities)}")

12584
============== Pattern 1 ==============
Average comprehensibility: 0.0
std comprehensibility: 0.0
var comprehensibility: 0.0
minimum comprehensibility: 0
maximum comprehensibility: 0
